In [1]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Product Dimension Table
categories = ['Dairy', 'Beverages', 'Snacks', 'Cleaning', 'Personal Care', 'Bakery', 'Frozen', 'Produce', 'Meat', 'Cereals']
subcategories = ['Organic', 'Gluten-Free', 'Low-Fat', 'Sugar-Free']
product_df = pd.DataFrame({
    'Product_ID': range(1, 101),
    'Product_Name': ['Product_' + str(i) for i in range(1, 101)],
    'Category': np.random.choice(categories, 100),
    'Subcategory': np.random.choice(subcategories, 100),
})

# Seasonality Dimension Table
date_range = pd.date_range(start='2020-01-01', end='2024-03-31')
seasonality_df = pd.DataFrame({
    'Date': date_range,
    'Day_of_Week': date_range.day_name(),
    'Month': date_range.month_name(),
    'Season': np.where(date_range.month % 12 < 3, 'Winter', 
                       np.where(date_range.month < 6, 'Spring', 
                                np.where(date_range.month < 9, 'Summer', 'Fall'))),
    'Holiday': np.random.choice([True, False], len(date_range), p=[0.1, 0.9]),
})

# Inventory Dimension Table
inventory_items_per_product = 5  # Each product has 5 inventory items
total_inventory_items = len(product_df) * inventory_items_per_product
inventory_dim_df = pd.DataFrame({
    'Inventory_ID': range(1, total_inventory_items + 1),
    'Product_ID': np.repeat(product_df['Product_ID'].values, inventory_items_per_product),
    'Restock_Frequency': np.random.choice(['Weekly', 'Bi-Weekly', 'Monthly'], total_inventory_items),
    'Lead_Time_Days': np.random.randint(1, 15, total_inventory_items),  # Simulated lead time in days for restocking
    #'Perishable': np.random.choice([True, False], total_inventory_items, p=[0.3, 0.7]),
})


In [2]:

df = pd.DataFrame(product_df)
# Mapping of categories to realistic subcategories
valid_mappings = {
   'Frozen': ['Sugar-Free', 'Gluten-Free', 'Organic', 'Low-Fat'],
   'Cleaning': ['NA'],  # No subcategories applicable
   'Produce': ['Organic'],
   'Personal Care': ['NA'],
   'Cereals': ['Sugar-Free', 'Gluten-Free', 'Organic', 'Low-Fat'],
   'Snacks': ['Sugar-Free', 'Gluten-Free', 'Organic', 'Low-Fat'],
   'Bakery': ['Sugar-Free', 'Gluten-Free', 'Organic', 'Low-Fat'],
   'Beverages': ['Sugar-Free', 'Gluten-Free', 'Organic', 'Low-Fat'],
   'Dairy': ['Sugar-Free', 'Gluten-Free', 'Organic', 'Low-Fat'],
   'Meat': [ 'Organic', 'Low-Fat']
}

def correct_subcategory(row):    
    row['Subcategory'] = np.random.choice(valid_mappings[row['Category']])
    return row
product_df = df.apply(correct_subcategory,axis = 1)


In [3]:
product_df

,Product_ID,Product_Name,Category,Subcategory
0,1,Product_1,Frozen,Sugar-Free
1,2,Product_2,Cleaning,NA
2,3,Product_3,Produce,Organic
3,4,Product_4,Personal Care,NA
4,5,Product_5,Frozen,Sugar-Free
...,...,...,...,...
95,96,Product_96,Cereals,Organic
96,97,Product_97,Meat,Organic
97,98,Product_98,Frozen,Low-Fat
98,99,Product_99,Meat,Low-Fat


In [4]:
# Base prices for each product within the specified range
base_prices = pd.Series(np.random.uniform(10, 1000, size=len(product_df)), index=product_df['Product_ID'])

# Fact Table: Daily Pricing and Sales Data
daily_data = pd.DataFrame({
    'Date': np.repeat(seasonality_df['Date'].values, len(product_df)),
    'Product_ID': np.tile(product_df['Product_ID'].values, len(seasonality_df)),
})

# Assigning consistent selling prices with minor daily fluctuations
daily_data['Base_Selling_Price'] = daily_data['Product_ID'].map(base_prices)
daily_variation = np.random.normal(1.0, 0.1, size=len(daily_data))  # Small percentage variation each day
daily_data['Selling_Price'] = daily_data['Base_Selling_Price'] * daily_variation

# Define a percentage range to reduce the selling price by, to set the cost price
# For example, cost price will be 40% to 90% of the selling price
cost_price_percentage_min = 0.4
cost_price_percentage_max = 0.9
# Generate a random percentage reduction within the defined range for each product
# This ensures variability but consistency for each product's cost price relative to its selling price
percentage_reduction = pd.Series(np.random.uniform(cost_price_percentage_min, cost_price_percentage_max, size=len(product_df)), index=product_df['Product_ID'])
# Apply the percentage reduction to the selling price to calculate the cost price
# The cost price for each product will now be consistently based on its selling price
daily_data['Cost_Price'] = daily_data.apply(lambda row: row['Selling_Price'] * percentage_reduction[row['Product_ID']], axis=1)

# Competitor prices close but varied around the selling price
daily_data['Competitor_Price'] = daily_data['Selling_Price'] * np.random.uniform(0.9, 1.1, size=len(daily_data))

# Function to adjust units sold based on conditions discussed
def units_sold(row):
    return np.random.randint(1000, 5000)  # Ensuring within the range

daily_data['Units_Sold'] = daily_data.apply(units_sold, axis=1)
daily_data['Revenue'] = daily_data['Units_Sold'] * daily_data['Selling_Price']


# Inventory Fact Table: Assuming inventory is adjusted based on sales
inventory_fact_df = pd.DataFrame({
    'Date': np.repeat(seasonality_df['Date'].values, total_inventory_items),
    'Inventory_ID': np.tile(inventory_dim_df['Inventory_ID'].values, len(seasonality_df)),
    'Product_ID': np.tile(inventory_dim_df['Product_ID'].values, len(seasonality_df))
})

# Assuming restocking to initial levels daily for simplicity
inventory_fact_df['Restocked_Units'] = np.where(inventory_fact_df['Date'].dt.dayofweek == 0,
                                                np.random.randint(100, 1000, size=len(inventory_fact_df)),
                                                0)

# Simulate daily sales deductions from inventory
inventory_fact_df['Daily_Sales'] = np.random.randint(10, 100, size=len(inventory_fact_df))

# Calculate net stock level change
inventory_fact_df['Net_Stock_Change'] = inventory_fact_df['Restocked_Units'] - inventory_fact_df['Daily_Sales']



In [6]:
daily_data

,Date,Product_ID,Base_Selling_Price,Selling_Price,Cost_Price,Competitor_Price,Units_Sold,Revenue
0,2020-01-01,1,753.666267,884.240079,408.923757,907.927103,2717,2.402480e+06
1,2020-01-01,2,80.297588,98.214390,65.065160,101.951610,3033,2.978842e+05
2,2020-01-01,3,89.378214,95.081001,46.196698,93.545203,3182,3.025477e+05
3,2020-01-01,4,361.223579,379.315262,287.059922,374.248792,4876,1.849541e+06
4,2020-01-01,5,942.308455,772.593262,517.610754,813.351221,3204,2.475389e+06
...,...,...,...,...,...,...,...,...
155195,2024-03-31,96,395.587076,367.318051,252.384756,377.566076,4410,1.619873e+06
155196,2024-03-31,97,391.656786,371.389970,269.624787,347.804248,3235,1.201447e+06
155197,2024-03-31,98,346.983233,407.383557,271.454566,392.971121,2747,1.119083e+06
155198,2024-03-31,99,545.993531,470.005605,406.318769,498.944865,2390,1.123313e+06


In [7]:
# Filter for the current year, assuming 'Date' is a datetime column
current_year_data = daily_data[daily_data['Date'].dt.year == 2024]  # Update the year as needed
# Calculate total units sold for each product in the current year
total_units_current_year = current_year_data.groupby('Product_ID')['Units_Sold'].sum()
# Get the top 5 products in the current year by units sold
top_5_products = total_units_current_year.nlargest(5).index.tolist()



# Update to adjust selling price for holidays and demand fluctuations
def adjust_selling_price(row):
   holiday_factor = 1.1 if row['Is_Holiday'] else 1.0
   weekend_factor = 1.05 if row['Is_Weekend'] else 1.0
   demand_fluctuation = np.random.uniform(0.95, 1.05)
   return row['Base_Selling_Price'] * holiday_factor * weekend_factor * demand_fluctuation
# Update to adjust units sold based on various factors
def adjust_units_sold(row):
   #price_sensitivity = -0.1  # Example sensitivity, adjust based on analysis
   holiday_boost = 1.5 if row['Is_Holiday'] else 1.0
   weekend_boost = 1.0
   if row['Product_ID'] in top_5_products and row['Is_Weekend'] and np.random.rand() <0.8:
        weekend_boost = 1.3
   price_factor = 1.0 if row['Selling_Price'] <= row['Competitor_Price'] else 0.5
   #price_effect = np.exp(price_sensitivity * (row['Selling_Price'] - row['Competitor_Price']))
   units_sold = row['Units_Sold'] * holiday_boost * weekend_boost * price_factor
   return max(0, np.round(units_sold))  # Ensure units sold is not negative

# Map day of week to weekend indicator
weekend_mapping = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 1, 'Sunday': 1}
seasonality_df['Is_Weekend'] = seasonality_df['Day_of_Week'].map(weekend_mapping)
# Add columns to indicate holidays and weekends
seasonality_df['Is_Holiday'] = seasonality_df['Holiday'].astype(int)



# Merge seasonality info into daily_data
daily_data = daily_data.merge(seasonality_df[['Date', 'Is_Holiday', 'Is_Weekend']], on='Date', how='left')
# Apply adjustments
daily_data['Selling_Price'] = daily_data.apply(adjust_selling_price, axis=1)
daily_data['Units_Sold'] = daily_data.apply(adjust_units_sold, axis=1)
daily_data['Revenue'] = daily_data['Units_Sold'] * daily_data['Selling_Price']
daily_data['Competitor_Price'] = daily_data['Selling_Price'] * np.random.uniform(0.9, 1.1, size=len(daily_data))

# Display a sample of the adjusted daily data
print(daily_data.sample(10))

             Date  Product_ID  Base_Selling_Price  Selling_Price  Cost_Price  \
53178  2021-06-15          79           40.646233      39.358745   36.376627   
137237 2023-10-04          38          791.095308     819.815688  683.730649   
11352  2020-04-23          53          436.224045     444.686787  200.609065   
137997 2023-10-11          98          346.983233     359.225218  177.629519   
95207  2022-08-10           8          368.300791     363.545391  167.341372   
107842 2022-12-14          43          917.468883     961.747892  302.898221   
53764  2021-06-21          65          680.349433     778.557758  602.298572   
135574 2023-09-17          75          510.400567     509.816220  252.724370   
76693  2022-02-05          94          295.521383     326.406732  182.294619   
71613  2021-12-17          14          874.961608     848.844547  372.867014   

        Competitor_Price  Units_Sold       Revenue  Is_Holiday  Is_Weekend  
53178          42.484983      3394.0  1.33

In [8]:
top_5_products

[36, 28, 15, 18, 1]

In [8]:
num_inventory_items={key:5 for key in range(1,101)}


In [9]:
inventory_fact_df

,Date,Inventory_ID,Product_ID,Restocked_Units,Daily_Sales,Net_Stock_Change
0,2020-01-01,1,1,0,10,-10
1,2020-01-01,2,1,0,26,-26
2,2020-01-01,3,1,0,75,-75
3,2020-01-01,4,1,0,68,-68
4,2020-01-01,5,1,0,81,-81
...,...,...,...,...,...,...
775995,2024-03-31,496,100,0,25,-25
775996,2024-03-31,497,100,0,78,-78
775997,2024-03-31,498,100,0,52,-52
775998,2024-03-31,499,100,0,47,-47


In [10]:
# First, ensure that the 'Units_Sold' from daily_data is merged into inventory_fact_df
inventory_fact_df = inventory_fact_df.merge(daily_data, on=['Date', 'Product_ID'], how='left')

# Initialize an empty list to hold the distributed sales data
distributed_sales = []

# Iterate over each group of items for the same product and date in the inventory fact DataFrame
for (date, product_id), group in inventory_fact_df.groupby(['Date', 'Product_ID']):
    items = num_inventory_items[product_id]
    total_sales = group['Units_Sold'].iloc[0]  # Total sales for this product on this date
    each_item_sales = total_sales // items  # Sales per inventory item
    remainder = total_sales % items  # Remainder to be distributed

    # Distribute sales across inventory items, adding the remainder to the last item
    sales_distribution = [each_item_sales] * items
    sales_distribution[-1] += remainder

    # Extend the distributed_sales list with the calculated sales distribution
    distributed_sales.extend(sales_distribution)

# Assign the distributed sales to the 'Daily_Sales' column
inventory_fact_df['Daily_Sales'] = distributed_sales

# Drop the now unnecessary 'Units_Sold' column
inventory_fact_df.drop(columns=['Units_Sold'], inplace=True)

# Display the updated inventory fact DataFrame
inventory_fact_df.head()


,Date,Inventory_ID,Product_ID,Restocked_Units,Daily_Sales,Net_Stock_Change,Base_Selling_Price,Selling_Price,Cost_Price,Competitor_Price,Revenue,Is_Holiday,Is_Weekend
0,2020-01-01,1,1,0,543.0,-10,753.666267,756.803162,408.923757,907.927103,2.056234e+06,0,0
1,2020-01-01,2,1,0,543.0,-26,753.666267,756.803162,408.923757,907.927103,2.056234e+06,0,0
2,2020-01-01,3,1,0,543.0,-75,753.666267,756.803162,408.923757,907.927103,2.056234e+06,0,0
3,2020-01-01,4,1,0,543.0,-68,753.666267,756.803162,408.923757,907.927103,2.056234e+06,0,0
4,2020-01-01,5,1,0,545.0,-81,753.666267,756.803162,408.923757,907.927103,2.056234e+06,0,0


In [11]:
def calculate_inventory(df, max_stock_per_product):
   df = df.sort_values(by='Date').reset_index(drop=True)  # Ensure chronological order and reset index
   # Initialize columns
   df['Restocked_Units'] = 0
   df['Net_Stock_Change'] = -df['Daily_Sales']  # Assuming sales reduce stock
   df['Running_Stock'] = 0
   for i in range(len(df)):
       if i < 500:  # First entry for the product's inventory items
           # Set initial stock levels based on max_stock_per_product
           df.at[i, 'Running_Stock'] = max_stock_per_product//5
       else:
           # Calculate restocked units based on sales and max stock constraint
           restock_needed = min(df.at[i, 'Daily_Sales'], max_stock_per_product - df.at[i-1, 'Running_Stock'])
           df.at[i, 'Restocked_Units'] = restock_needed
           # Update Net_Stock_Change to reflect restocking
           df.at[i, 'Net_Stock_Change'] = restock_needed - df.at[i, 'Daily_Sales']
           # Update Running_Stock
           df.at[i, 'Running_Stock'] = df.at[i-1, 'Running_Stock'] + df.at[i, 'Net_Stock_Change']
           # Ensure Running_Stock is within [0, max_stock_per_product]
           df.at[i, 'Running_Stock'] = max(min(df.at[i, 'Running_Stock'], max_stock_per_product), 0)
   return df

inventory_fact_df = inventory_fact_df.groupby('Product_ID').apply(calculate_inventory, max_stock_per_product=2000).reset_index(drop=True)

In [12]:
# Seed for reproducibility
np.random.seed(42)

# Campaign details
n_campaigns = 100
campaign_types = ['Email Blast', 'Social Media Ad', 'Holiday Sale', 'Limited Time Offer', 'New Product Launch']
discount_types = ['Percentage Off', 'Buy One Get One Free', 'Fixed Amount Off']
product_categories = ['Dairy', 'Beverages', 'Snacks', 'Cleaning', 'Personal Care', 'Bakery', 'Frozen', 'Produce', 'Meat', 'Cereals']

# Campaign DataFrame
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2024-12-31')
date_range = (end_date - start_date).days

campaign_df = pd.DataFrame({
    'Campaign_ID': range(1, n_campaigns + 1),
    'Campaign_Type': np.random.choice(campaign_types, n_campaigns),
    'Discount_Type': np.random.choice(discount_types, n_campaigns),
    'Targeted_Category': np.random.choice(product_categories, n_campaigns),
    'Start_Date': start_date + pd.to_timedelta(np.random.randint(0, date_range - 30, size=n_campaigns), unit='D'),
})


# Assign End_Date, ensuring it's after Start_Date
campaign_df['End_Date'] = campaign_df['Start_Date'] + pd.to_timedelta(np.random.randint(1, 30, size=n_campaigns), unit='D')
campaign_df['End_Date'] = campaign_df.apply(lambda row: row['Start_Date'] + pd.DateOffset(days=1) if row['End_Date'] < row['Start_Date'] else row['End_Date'], axis=1)

print(campaign_df)
campaign_df.to_csv('campaign_data.csv')


    Campaign_ID       Campaign_Type         Discount_Type Targeted_Category  \
0             1  Limited Time Offer      Fixed Amount Off             Dairy   
1             2  New Product Launch        Percentage Off         Beverages   
2             3        Holiday Sale  Buy One Get One Free         Beverages   
3             4  New Product Launch        Percentage Off            Bakery   
4             5  New Product Launch      Fixed Amount Off            Frozen   
..          ...                 ...                   ...               ...   
95           96        Holiday Sale        Percentage Off           Produce   
96           97  New Product Launch        Percentage Off           Produce   
97           98     Social Media Ad        Percentage Off            Frozen   
98           99     Social Media Ad        Percentage Off            Snacks   
99          100         Email Blast      Fixed Amount Off             Dairy   

   Start_Date   End_Date  
0  2021-10-01 2021-10-15

In [13]:
daily_data.to_csv('daily_sales_data.csv')
campaign_df.to_csv('campaign_data.csv')
inventory_dim_df.to_csv('inventory_dim.csv')
inventory_fact_df.to_csv('inventory_fact.csv')
product_df.to_csv('product_dim.csv')
seasonality_df.to_csv('seasonality_dim.csv')